# Local linear notebook


* [local-linear-visualize]: visualize the linear segments over a 2D space
* [local-linear-size]: compute the typical size of local linear segments
* [local-linear-satisfy]: do the points of the local linear segments satisfy the condi-
tions of root points?

In [ ]:
%env CUDA_VISIBLE_DEVICES=""
%load_ext autoreload
%autoreload 2

from typing import Union, Callable

import seaborn.apionly as sns
import numpy as np
import torch 
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd
import savethat

from lrp_relations import local_linear, dtd, utils

## [local-linear-visualize]: visualize the linear segments over a 2D space

In [ ]:
# run sampling

utils.set_project_dir()
sample_grid = savethat.create_node(
    local_linear.SampleGrid,
    args=local_linear.SampleGridArgs(),
    credentials=utils.get_credentials(),
)
grid = sample_grid.run()


In [ ]:
grid.grads_logit_0.shape

In [ ]:
grid_np = grid.grid.detach().numpy()
for i in range(2):
    plt.scatter(
        grid_np[:, 0],
        grid_np[:, 1],
        # c=net(inputs).argmax(dim=1).numpy(),
        # c=net(inputs)[:, 0].detach().numpy(),
        c=grid.logits[:, i].detach().numpy(),
    )
    plt.colorbar()
    plt.title(f"Logit {i}")
    plt.show()

In [ ]:

colors = np.array(sns.color_palette("colorblind", 100))
# print(colors)

for grad in [grid.grads_logit_0, grid.grads_logit_1]:
    grad_colors = dtd.almost_unique(grad, atol=1e-4)
    # print(grad_colors.unique().shape)
    plt.scatter(
        grid_np[:, 0],
        grid_np[:, 1],
        c=colors[grad_colors.numpy() % len(colors)],
        marker='.',
        s=1,
    )
    plt.show()

In [ ]:
torch.manual_seed(0)

in_channels = 2
x = torch.rand(1, in_channels).repeat(40, 1)
mlp = dtd.NLayerMLP(5, in_channels, 20, 1)

chain, accepts = local_linear.sample(mlp, x, n_steps=200, n_warmup=0)
